# Helm 201

A deep-dive into Helm (v3) and details like

* Templating
* Charts and Subcharts
* Usage and internal structure in Kubernetes
* Integrations

In [76]:
wd_init = "work/helm-init2"



In [77]:
!helm version


version.BuildInfo{Version:"v3.8.0", GitCommit:"d14138609b01886f544b2025f5000351c9eb092e", GitTreeState:"clean", GoVersion:"go1.17.6"}


---
---

## Init

* Create a new template / Helm Chart

In [78]:
!echo $wd_init
!mkdir -p $wd_init

!helm create $wd_init/demo-helm-201

!tree $wd_init

work/helm-init2
Creating work/helm-init2/demo-helm-201
work/helm-init2
└── demo-helm-201
    ├── Chart.yaml
    ├── charts
    ├── templates
    │   ├── NOTES.txt
    │   ├── _helpers.tpl
    │   ├── deployment.yaml
    │   ├── hpa.yaml
    │   ├── ingress.yaml
    │   ├── service.yaml
    │   ├── serviceaccount.yaml
    │   └── tests
    │       └── test-connection.yaml
    └── values.yaml

4 directories, 10 files


## Chart Structure and Overview

* `Chart.yaml` common meta information
  * avoid using `appVersion` - handle version in env-specific value-files or as field which will be overwritten during helm execution

In [80]:
!cat $wd_init/demo-helm-201/Chart.yaml | grep -B2 -i 'version:'

apiVersion: v2
--
# to the chart and its templates, including the app version.
# Versions are expected to follow Semantic Versioning (https://semver.org/)
version: 0.1.0
--
# follow Semantic Versioning. They should reflect the version the application is using.
# It is recommended to use it with quotes.
appVersion: "1.16.0"


----
* Templating
  * include output/result from namespaced functions
  * Whitespaces and new lines, indent


* Action (00)
  * modify service template

In [85]:
!echo "Render template and generate Kubernetes resource files"

!helm template demo-helm-201-common $wd_init/demo-helm-201 -f $wd_init/demo-helm-201/values.yaml --output-dir=work/out/common

Render template and generate Kubernetes resource files
wrote work/out/common/demo-helm-201/templates/serviceaccount.yaml
wrote work/out/common/demo-helm-201/templates/service.yaml
wrote work/out/common/demo-helm-201/templates/deployment.yaml
wrote work/out/common/demo-helm-201/templates/tests/test-connection.yaml



----
* Templating, Variables
  * `.Values.*` holds all variables (from file and command line)
  * `values.yaml` and any additional values-file will be merged

 


* Action (01)
  * `image.version`
  * env-specific values file


In [84]:
!echo "Render template and generate Kubernetes resource files for *TEST* stage"

!helm template demo-helm-201-test $wd_init/demo-helm-201 -f $wd_init/demo-helm-201/values.test.yaml --output-dir=work/out/test

Render template and generate Kubernetes resource files for *TEST* stage
wrote work/out/test/demo-helm-201/templates/serviceaccount.yaml
wrote work/out/test/demo-helm-201/templates/service.yaml
wrote work/out/test/demo-helm-201/templates/deployment.yaml
wrote work/out/test/demo-helm-201/templates/tests/test-connection.yaml



---

* Templating, Functions
  * `_helpers.tpl` holds set of helper functions used in template files
  * common Go template functions are included (and, or, len, ...)

----
* Templating, Functions
  * `with` set the scope
  * `range` iterate

* Action (02)
  * new `sa.yaml`
  * values in `values.test.yaml`

In [87]:
!echo "Render template and generate Kubernetes resource files for *TEST* stage"

!helm template demo-helm-201-test $wd_init/demo-helm-201 -f $wd_init/demo-helm-201/values.test.yaml --output-dir=work/out/test

Render template and generate Kubernetes resource files for *TEST* stage
wrote work/out/test/demo-helm-201/templates/02_sa.yaml
wrote work/out/test/demo-helm-201/templates/02_sa.yaml
wrote work/out/test/demo-helm-201/templates/serviceaccount.yaml
wrote work/out/test/demo-helm-201/templates/service.yaml
wrote work/out/test/demo-helm-201/templates/deployment.yaml
wrote work/out/test/demo-helm-201/templates/tests/test-connection.yaml



----
* Charts and Subcharts
  * Subchart is a standalone chart
  * only parent knows the subcharts / childrens
  * ...and only parent can override fields

* Action 
  * checkout existing chart
  * create a new chart in `parent/chart` directory
  * configure dependency

In [88]:
!echo "Print out existing chart (without subchart)"

!tree $wd_init

Print out existing chart (without subchart)
work/helm-init2
└── demo-helm-201
    ├── Chart.yaml
    ├── charts
    ├── templates
    │   ├── 02_sa.yaml
    │   ├── NOTES.txt
    │   ├── _helpers.tpl
    │   ├── deployment.yaml
    │   ├── hpa.yaml
    │   ├── ingress.yaml
    │   ├── service.yaml
    │   ├── serviceaccount.yaml
    │   └── tests
    │       └── test-connection.yaml
    ├── values.test.yaml
    └── values.yaml

4 directories, 12 files


In [89]:
!echo "Create new (sub)chart in the *charts* dir"

!helm create $wd_init/demo-helm-201/charts/demo-subchart
!rm -rf $wd_init/demo-helm-201/charts/demo-subchart/templates/*

!tree $wd_init

Create new (sub)chart in the *charts* dir
Creating work/helm-init2/demo-helm-201/charts/demo-subchart
work/helm-init2
└── demo-helm-201
    ├── Chart.yaml
    ├── charts
    │   └── demo-subchart
    │       ├── Chart.yaml
    │       ├── charts
    │       ├── templates
    │       └── values.yaml
    ├── templates
    │   ├── 02_sa.yaml
    │   ├── NOTES.txt
    │   ├── _helpers.tpl
    │   ├── deployment.yaml
    │   ├── hpa.yaml
    │   ├── ingress.yaml
    │   ├── service.yaml
    │   ├── serviceaccount.yaml
    │   └── tests
    │       └── test-connection.yaml
    ├── values.test.yaml
    └── values.yaml

7 directories, 14 files


* Action (03)
  * template for `ConfigMap` in subchart
  * value file in subchart
  * override options

In [90]:
!echo "Dry-Run subchart - with local value file"

!helm install --generate-name --dry-run --debug $wd_init/demo-helm-201/charts/demo-subchart

Dry-Run subchart - with local value file
install.go:178: [debug] Original chart version: ""
install.go:195: [debug] CHART PATH: /Users/haddouti/repos/haf-tech/helm-201/work/helm-init2/demo-helm-201/charts/demo-subchart

I0206 12:25:30.739877   62851 request.go:665] Waited for 1.116660086s due to client-side throttling, not priority and fairness, request: GET:https://c113-e.eu-de.containers.cloud.ibm.com:30919/apis/kubeflow.org/v1alpha1?timeout=32s
NAME: demo-subchart-1644146728
LAST DEPLOYED: Sun Feb  6 12:25:36 2022
NAMESPACE: demo-helm
STATUS: pending-install
REVISION: 1
TEST SUITE: None
USER-SUPPLIED VALUES:
{}

COMPUTED VALUES:
commonItem: mega-one

HOOKS:
MANIFEST:
---
# Source: demo-subchart/templates/03_configmap.yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: demo-subchart-1644146728-cm
data:
  item: mega-one



In [91]:
!echo "Render entire chart - with *pre* value file, to override subchart fields"

!helm template demo-helm-201-pre $wd_init/demo-helm-201 -f $wd_init/demo-helm-201/values.pre.yaml --output-dir=work/out/pre

Render entire chart - with *pre* value file, to override subchart fields
wrote work/out/pre/demo-helm-201/templates/02_sa.yaml
wrote work/out/pre/demo-helm-201/templates/02_sa.yaml
wrote work/out/pre/demo-helm-201/templates/serviceaccount.yaml
wrote work/out/pre/demo-helm-201/charts/demo-subchart/templates/03_configmap.yaml
wrote work/out/pre/demo-helm-201/templates/service.yaml
wrote work/out/pre/demo-helm-201/templates/deployment.yaml
wrote work/out/pre/demo-helm-201/templates/tests/test-connection.yaml



In [92]:
!echo "Render entire chart - with *test* value file, to *NOT* override subchart fields"

!helm template demo-helm-201-test $wd_init/demo-helm-201 -f $wd_init/demo-helm-201/values.test.yaml --output-dir=work/out/test

Render entire chart - with *test* value file, to *NOT* override subchart fields
wrote work/out/test/demo-helm-201/templates/02_sa.yaml
wrote work/out/test/demo-helm-201/templates/02_sa.yaml
wrote work/out/test/demo-helm-201/templates/serviceaccount.yaml
wrote work/out/test/demo-helm-201/charts/demo-subchart/templates/03_configmap.yaml
wrote work/out/test/demo-helm-201/templates/service.yaml
wrote work/out/test/demo-helm-201/templates/deployment.yaml
wrote work/out/test/demo-helm-201/templates/tests/test-connection.yaml



---
---

## Life Cycle and Hooks

Hooks allows to execute specific resource definitions at defined points in the Helm life cycle.
Available hooks

* `pre-install`
* `post-install`
* `pre-delete`
* `post-delete`
* `pre-upgrade`
* `post-upgrade`
* `pre-rollback`
* `post-rollback`
* `test`

Examples

* prepare the installation and create specific resources beforehand (ConfigMap, Job etc)
* clean up database before uninstalling application
* return a license or deregister/unsubscribe


### Pre-Install and Post-Delete

Example for `pre-install` and `post-delete` hook. Hooks are usual Kubernetes resource definitions with special annotations.

```
metadata:
  annotations:
    "helm.sh/hook": pre-install, post-delete
    "helm.sh/hook-weight": "-5"
    "helm.sh/hook-delete-policy": hook-succeeded
```

In [93]:
!echo "Create new (sub)chart in the *charts* dir"

!helm create $wd_init/demo-helm-201/charts/demo-hook
!rm -rf $wd_init/demo-helm-201/charts/demo-hook/templates/ingress.yaml
!rm -rf $wd_init/demo-helm-201/charts/demo-hook/templates/service.yaml
!rm -rf $wd_init/demo-helm-201/charts/demo-hook/templates/tests


!tree $wd_init

Create new (sub)chart in the *charts* dir
Creating work/helm-init2/demo-helm-201/charts/demo-hook
work/helm-init2
└── demo-helm-201
    ├── Chart.yaml
    ├── charts
    │   ├── demo-hook
    │   │   ├── Chart.yaml
    │   │   ├── charts
    │   │   ├── templates
    │   │   │   ├── NOTES.txt
    │   │   │   ├── _helpers.tpl
    │   │   │   ├── deployment.yaml
    │   │   │   ├── hpa.yaml
    │   │   │   └── serviceaccount.yaml
    │   │   └── values.yaml
    │   └── demo-subchart
    │       ├── Chart.yaml
    │       ├── charts
    │       ├── templates
    │       │   └── 03_configmap.yaml
    │       └── values.yaml
    ├── templates
    │   ├── 02_sa.yaml
    │   ├── NOTES.txt
    │   ├── _helpers.tpl
    │   ├── deployment.yaml
    │   ├── hpa.yaml
    │   ├── ingress.yaml
    │   ├── service.yaml
    │   ├── serviceaccount.yaml
    │   └── tests
    │       └── test-connection.yaml
    ├── values.pre.yaml
    ├── values.test.yaml
    └── values.yaml

10 directories, 23 files


---

* Action (04)
  * create 2 jobs for different hooks
  * install and upgrade

In [95]:
!echo "Install subchart for hook testing"

!echo "some OCP permissions adjustments (for nginx)..."
!oc adm policy add-scc-to-user anyuid -z demo-hook-dev

!helm upgrade --install demo-hook-dev $wd_init/demo-helm-201/charts/demo-hook -n demo-helm

Install subchart for hook testing
some OCP permissions adjustments (for nginx)...
clusterrole.rbac.authorization.k8s.io/system:openshift:scc:anyuid added: "demo-hook-dev"
Release "demo-hook-dev" does not exist. Installing it now.
I0206 12:37:58.365110   65109 request.go:665] Waited for 1.056190148s due to client-side throttling, not priority and fairness, request: GET:https://c113-e.eu-de.containers.cloud.ibm.com:30919/apis/apiserver.openshift.io/v1?timeout=32s
NAME: demo-hook-dev
LAST DEPLOYED: Sun Feb  6 12:38:04 2022
NAMESPACE: demo-helm
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
1. Get the application URL by running these commands:
  export POD_NAME=$(kubectl get pods --namespace demo-helm -l "app.kubernetes.io/name=demo-hook,app.kubernetes.io/instance=demo-hook-dev" -o jsonpath="{.items[0].metadata.name}")
  export CONTAINER_PORT=$(kubectl get pod --namespace demo-helm $POD_NAME -o jsonpath="{.spec.containers[0].ports[0].containerPort}")
  echo "Visit http://127.0.0.1:80

In [96]:
!echo "Update subchart for hook testing now with job definitions"

!helm upgrade --install demo-hook-dev $wd_init/demo-helm-201/charts/demo-hook -n demo-helm

Update subchart for hook testing now with job definitions
I0206 12:39:11.932426   66142 request.go:665] Waited for 1.149813455s due to client-side throttling, not priority and fairness, request: GET:https://c113-e.eu-de.containers.cloud.ibm.com:30919/apis/template.openshift.io/v1?timeout=32s
Release "demo-hook-dev" has been upgraded. Happy Helming!
NAME: demo-hook-dev
LAST DEPLOYED: Sun Feb  6 12:39:17 2022
NAMESPACE: demo-helm
STATUS: deployed
REVISION: 2
TEST SUITE: None
NOTES:
1. Get the application URL by running these commands:
  export POD_NAME=$(kubectl get pods --namespace demo-helm -l "app.kubernetes.io/name=demo-hook,app.kubernetes.io/instance=demo-hook-dev" -o jsonpath="{.items[0].metadata.name}")
  export CONTAINER_PORT=$(kubectl get pod --namespace demo-helm $POD_NAME -o jsonpath="{.spec.containers[0].ports[0].containerPort}")
  echo "Visit http://127.0.0.1:8080 to use your application"
  kubectl --namespace demo-helm port-forward $POD_NAME 8080:$CONTAINER_PORT


---

* Action
  * current `pods` and `jobs`
  * events
  * clean-up and check out the job with `post-delete`

In [97]:
!echo "Delete subchart which triggers hook"

!helm delete demo-hook-dev -n demo-helm

Delete subchart which triggers hook
I0206 12:40:33.899998   67391 request.go:665] Waited for 1.146978078s due to client-side throttling, not priority and fairness, request: GET:https://c113-e.eu-de.containers.cloud.ibm.com:30919/apis/helm.openshift.io/v1beta1?timeout=32s
release "demo-hook-dev" uninstalled


---
---

## OCI Registry

Helm 3 provides the option to store helm charts not only on a HTTP server, but also in a OCI Registry. This allows the option to hold the artifacts (container image and helm charts) in the same registry.

### Testing

* Action
  * use a local registry
  * push a local helm chart
  * install from OCI registry

  

In [98]:
!echo "Run a local registry"

!docker run -dp 5000:5000 --restart=always --name registry registry

Run a local registry
4a9a3d4acdc7fc793ba4b0e0638dbbc818ae2f8919d5577509f34858e122e15f


In [99]:
!echo "Create helm package from the hook subchart"

!echo "...copy subchart in own dir for testing..."
!cp -r $wd_init/demo-helm-201/charts/demo-hook $wd_init/

!echo "...helm package..."
!helm package $wd_init/demo-hook --version 1.0.3 --destination $wd_init
!helm package $wd_init/demo-hook --version 1.0.4 --destination $wd_init

Create helm package from the hook subchart
...copy subchart in own dir for testing...
...helm package...
Successfully packaged chart and saved it to: work/helm-init2/demo-hook-1.0.3.tgz
Successfully packaged chart and saved it to: work/helm-init2/demo-hook-1.0.4.tgz


In [100]:
!echo "Push helm package to registry"

!echo "...helm registry login..."
!helm registry login -u testuser -p testpassword localhost:5000

!echo "...helm package push..."
!helm push $wd_init/demo-hook-1.0.3.tgz oci://localhost:5000/helm-charts
!helm push $wd_init/demo-hook-1.0.4.tgz oci://localhost:5000/helm-charts

Push helm package to registry
...helm registry login...
INFO[0000] Error logging in to endpoint, trying next endpoint  error="Get \"https://localhost:5000/v2/\": http: server gave HTTP response to HTTPS client"
Login Succeeded
...helm package push...
Pushed: localhost:5000/helm-charts/demo-hook:1.0.3
Digest: sha256:bb0aac66e5d7ad67d87bc36e7cc754972b4c4249de9e175e3de2f6b5f38b5d07
Pushed: localhost:5000/helm-charts/demo-hook:1.0.4
Digest: sha256:a7e6e3b0a44a2fc23395c57d7f5d9046f59e5e4bc29e1610df34d247d5fcdc62


In [101]:
!echo "Verify helm package in OCI registry"

!echo "...show chart details..."
!helm show chart oci://localhost:5000/helm-charts/demo-hook


Verify helm package in OCI registry
...show chart details...
apiVersion: v2
appVersion: 1.16.0
description: A Helm chart for Kubernetes
name: demo-hook
type: application
version: 1.0.4



In [103]:
!echo "Install (render) helm package directly from OCI registry"

!echo "...render existing chart from registry..."
!helm template release-demo-hook-oci oci://localhost:5000/helm-charts/demo-hook --version 1.0.3 --output-dir=work/out/oci

!echo "...try to render non-existing chart from registry..."
!helm template release-demo-hook-oci oci://localhost:5000/helm-charts/demo-hook --version 2.2.2 --output-dir=work/out/oci


Install (render) helm package directly from OCI registry
...render existing chart from registry...
wrote work/out/oci/demo-hook/templates/serviceaccount.yaml
wrote work/out/oci/demo-hook/templates/deployment.yaml
wrote work/out/oci/demo-hook/templates/04_job.yaml
wrote work/out/oci/demo-hook/templates/04_job.yaml
wrote work/out/oci/demo-hook/templates/04_job.yaml

...try to render non-existing chart from registry...
Error: failed to download "oci://localhost:5000/helm-charts/demo-hook" at version "2.2.2"


---
---

## Misc

Some additional details


### Internal/Release State

With Helm3 not Tiller exists anymore. All configuration and release state, including value files, is now stored in a `Secret` instead of a `ConfigMap`.


In [ ]:
!echo "Install subchart"

!helm upgrade --install release-demo-hook-state $wd_init/demo-helm-201/charts/demo-hook -n demo-helm

In [ ]:
!echo "List all existing helm releases"

!helm list

In [ ]:
!echo "Print out release information from Helm Release Secret"

!oc get secret sh.helm.release.v1.release-demo-hook-state.v1 -o jsonpath='{.data.release}' | base64 --decode | base64 --decode | gunzip -c | jq

### Helm Upgrade and force re-deployment

Helm determines the relevant changes and updates only the necessary resources. In case the content of a `ConfigMap` changed, but not the image version, a redeployment or restart of the deployment will be not triggered.
To trigger a redeployment a relation between `ConfigMap` content and `Deployment` is needed

```
kind: Deployment
spec:
  template:
    metadata:
      annotations:
        checksum/config: {{ include (print $.Template.BasePath "/configmap.yaml") . | sha256sum }}
```

The idea is, that the `Deployment` contains an annotation with a checksum of the related `ConfigMap`. Any changes in the content, results in an other `sha256sum`, results in an other annotation, which results in modified `Deployment` definition.

### VS Code Extensions

* `yaml`
* `kubernetes` understands also the helm template syntax

## Summary

This was a walkthrough for Helm 201 with covering topics like

* templating
* charts and subcharts
* OCI Registry

## References

* [Helm - Storage Provider - Secret instead ConfigMap](https://helm.sh/docs/topics/advanced/#storage-backends)
* [Helm - (OCI) Registry](https://helm.sh/docs/topics/registries/)
* [Helm - Best Practices](https://helm.sh/docs/chart_best_practices/conventions/)
* [Helm - Template Guide](https://helm.sh/docs/chart_template_guide/getting_started/)